In [3]:
from fastai.vision import *

In [29]:
from pathlib import Path
import pandas as pd

DATA_ROOT = Path('./covid-chestxray-dataset')
DATA_IMAGES = DATA_ROOT / 'images'

df = pd.read_csv(DATA_ROOT / 'metadata.csv')

# drop useless cols
drop_cols = [
    'needed_supplemental_O2', 
    'extubated',
    'temperature',
    'pO2_saturation',
    'leukocyte_count',
    'neutrophil_count',
    'lymphocyte_count'
]
df.drop(drop_cols, axis=1, inplace=True)

# we need only these cols
cols = [
    'patientid',
    'sex',
    'age',
    'finding',
    'folder',
    'filename',
]
df = df[cols]

df['path'] = df['folder'] + '/' + df['filename']

# also drop all NaN values
df.dropna(inplace=True)
df.count()

patientid    500
sex          500
age          500
finding      500
folder       500
filename     500
path         500
dtype: int64

In [30]:
dataBunch = ImageDataBunch.from_df(DATA_ROOT, df, fn_col='path', label_col='finding')

You can deactivate this warning by passing `no_check=True`.


/usr/local/lib/python3.7/dist-packages/fastai/basic_data.py:262: UserWarning: There seems to be something wrong with your dataset, for example, in the first batch can't access these elements in self.train_ds: 156,173
  warn(warn_msg)


In [31]:
dataBunch.show_batch()

FileNotFoundError: [Errno 2] No such file or directory: 'covid-chestxray-dataset/volumes/radiopaedia_org_covid-19-pneumonia-40_86625_0-dcm.nii.gz'